In [ ]:
import pandas as pd
import json

In [ ]:
df = pd.read_csv('comments_procces.csv')

columns_to_drop = ['lead_time', 'updated_at', 'created_at', "annotator"]
df = df.drop(columns=columns_to_drop, errors='ignore')

display(df.head())

In [ ]:
print("Общая информация о данных:")
df.info()

In [ ]:
categories = [
    "Нравится скорость отработки заявок",
    "Нравится качество выполнения заявки",
    "Нравится качество работы сотрудников",
    "Понравилось выполнение заявки",
    "Вопрос решен",
    "Другое"
]

for category in categories:
    if category not in df.columns:
        df[category] = 0

def extract_categories(taxonomy_str):
    try:
        data = json.loads(taxonomy_str)
        cats = []
        if isinstance(data, list):
            for item in data:
                if isinstance(item, dict) and 'taxonomy' in item:
                    nested = item['taxonomy']
                    if isinstance(nested, list):
                        for sublist in nested:
                            if isinstance(sublist, list):
                                cats.extend(sublist)
        return [cat.strip() for cat in cats]
    except:
        return []

for idx, row in df.iterrows():
    cats_in_cell = extract_categories(row['taxonomy'])
    for category in categories:
        df.at[idx, category] = 1 if category in cats_in_cell else 0

print("Обработка завершена.")

In [ ]:
df = df.drop_duplicates(subset=['comment'])

if 'taxonomy' in df.columns:
    df = df.drop(columns=['taxonomy'])

In [ ]:
df = pd.read_csv('comments_processed_time.csv')

columns = [
    'Нравится скорость отработки заявок',
    'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Понравилось выполнение заявки',
    'Вопрос решен',
    'Другое'
]

for col in columns:
    if col in df.columns:
        count_ones = (df[col] == 1).sum()
        print(f"Статистика по столбцу: {col}: {count_ones}")
    else:
        print(f"Столбец '{col}' не найден в файле.")